In [1]:
%load_ext autoreload

In [2]:
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/Github/Abstract-generator/notebooks

In [3]:
%%capture
if IN_COLAB:
  !pip install feedparser tokenizers transformers scipy==1.7.1;

In [4]:
import os, torch, time, math, sys, re, csv
import numpy as np

PACKAGE_ROOT = os.path.dirname(os.path.abspath(''))
print(PACKAGE_ROOT)
sys.path.append(PACKAGE_ROOT)

from src import settings
import src.data.dataset_class as dsc
import src.data.dataloader_class as dlc

from src.model.transformer_torch import TransformerModel
from src.model.generate_text import gen_some_text

from src.model.train_evaluate import train_version_jeremy as train
from src.model.train_evaluate import evaluate_version_jeremy as evaluate

#from src.model.transformer import make_gpt_model # imports don't work

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jbrothschild/Documents/HLML/Abstract-generator


### Parameters

In [5]:
# ARCHITECTURE
max_len_sentence     = 40 # maximum sentence length
vocab_size  = None # None if you want to let tokenizer do its thing
emsize     = 512 # embedding dimension
nhid       = 2048 # the dimension of the feedforward network model in torch.nn.TransformerEncoder
nlayers    = 12 # the number of torch.nn.TransformerEncoderLayer in torch.nn.TransformerEncoder
nhead      = 8 # the number of heads in the multiheadattention models
dropout    = 0.2 # the dropout value
batch_size = 10 #32
val_batch_size = 10 #32, not used right now.
epochs     = 50  # The number of epochs

TRAIN = True

### Format Dataset

Uses a custom dataset class, which is an iterable and callable structure that returns a sample from our dataset. Within this custom dataset, can determine all preprocessing.

In [6]:
# create dataset
dataset = dsc.ArxivDataset()
#dataset = dsc.WikiTextDataset()

#train tokenizer (or use one already trained)
tknzr_type = 'BPE'
flag_tknzr_train = True
flag_tknzr_fast = True

_ = dataset.tokenizer(flag_tknzr_train, tknzr_type, flag_tknzr_fast=flag_tknzr_fast)

### Creating DataLoaders

Training is done on batches, so we need a way to extract groupings of the data in the appropriate format for our transformer model.
Note that for transformers which we are training, dataloaders outputs both src (x[:-1] and tgt ([1:]).
The collation of batches for different transformer models we have vary. For HuggingFace it's ( max_len x batch_size ) whereas I think that the Annotated Transformer has ( batch_size x max_len ).

I created a custom Dataloader class that wraps splitting the dataset and also outputs different dataloaders for each.

NOTE : Do not use the tokenizer before the training if you use num_workers>0!
FastTokenizer does not play nicely with forking if you use it before the forking of your data:
https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning

In [7]:
dataloader = dlc.CustomDataloader(dataset, batch_size, max_len_sentence)

### Selecting model

Here we choose which model we shall use for training. For now, I've selected the black box Transformer from HuggingFace because the collate_fn I've written gives the correct input size force it... however this can easily be changed! 

In [19]:
# transformer from huggingface
# TODO : Change to the Annotated Transformer if I want
model = TransformerModel(dataset.vocab_size, emsize, nhead, nhid, nlayers, dropout).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()#ignore_index=tknzr.get_vocab()["<pad>"])

# optimizer
paramsAdam  = [{'params' : model.parameters(), 'lr' : 1e-3, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsAdamW = [{'params' : model.parameters(), 'lr' : 5e-5, 'betas' : (0.9, 0.999), 'eps' : 1e-08, 'weight_decay' : 0.0}]
paramsSGD   = [{'params' : model.parameters(), 'lr' : 0.5, 'momentum' : 0.0, 'dampening' : 0.0, 'weight_decay' : 0.0}]

optimizer = torch.optim.SGD( paramsSGD )
#optimizer = torch.optim.Adam( paramsAdam )
#optimizer = torch.optim.AdamW( paramsAdamW )

# scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95) # 1.0 to signify no decay rate

### Training

Training loop!

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # fasttokenizer should not be used before forking. Something
                                                # to figure out. What this does is suppress some warning messages 
                                                # https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning
                                                # doesn't seem to affect the timing though
if TRAIN:
    best_val_loss = float("inf")
    best_model = None
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model, dataloader.train, device, dataset.vocab_size, epoch, optimizer, scheduler, criterion, max_len_sentence)
        val_loss = evaluate(model, dataloader.valid, device, dataset.vocab_size, criterion, max_len_sentence, len(dataloader.dataset_valid))
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
              'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                         val_loss, math.exp(val_loss)))
                                         # Why is math.exp so large????
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

        scheduler.step()

    # save best model (two methods)
    model_full = settings.DIR_MODELS + os.sep + f'{dataset.name}_epoch{epochs}.pth'
    model_weights = settings.DIR_MODELS + os.sep + f'{dataset.name}_weights_epoch{epochs}.pth'
    model_full_best = settings.DIR_MODELS + os.sep + f'{dataset.name}_epoch{epochs}_best.pth'
    model_weights_best = settings.DIR_MODELS + os.sep + f'{dataset.name}_weights_epoch{epochs}_best.pth'
    # approach 1: save model (class) entirely (uses pickle)
    torch.save(model, model_full)
    torch.save(best_model, model_full_best)
    # approach 2: save model weights
    torch.save(best_model.state_dict(), model_weights_best)

700


### Text Generation

Here I've simply taken the code Matt uses to generate text.

In [ ]:
if not TRAIN:
    custom_filename = 'arxiv_10000'
    custom_epochs = 10
    model_full = settings.DIR_MODELS + os.sep + f'{custom_filename}_epoch{custom_epochs}_best.pth'
    model_weights = settings.DIR_MODELS + os.sep + f'{custom_filename}_weights_epoch{custom_epochs}_best.pth'
    
    # approach 1: load model (class) entirely (uses pickle)
    model_full_load = torch.load(model_full, map_location=device)

    # approach 2: load model weights, need to have some parameter or something 
    model_load = TransformerModel(vocab_size, emsize, nhead, nhid, nlayers, dropout).to(device)
    model_weights_load = model_load.load_state_dict( torch.load(model_weights) )

In [10]:
# inspect both models
#print('model_A info...\n', model_full_load)
#print('\nmodel_B info...\n', model_weights_load)

#print('model_A == model_B:', model_full_load == model_weights_load)
#model = model_full_load
# Text generation example

#model = model_load
prompt = 'The dog ran'
ngen = 100
decode_style = 'sample_topp' #greedy, sample_topp
model.to('cpu')
generated_text = gen_some_text(
    best_model, dataset.transform, 'cpu', max_len_sentence, text_prompt=prompt, tokens_to_gen=ngen, vis=False,
    decode_style=decode_style)
print("Text prompt:\n", prompt)
print("Number of tokens to generate:", ngen)
print("Generated_text:\n", generated_text)

# TODO: alternative generation
# currently 'greedy method'
# see: https://huggingface.co/blog/how-to-generate

TypeError: ignored